# Defining Substorm Epoch
In this notebook we will define substorm epoch for each substorm list creating new columns in for each substorm list in the AKR flimits file created in the Data_Download.ipynb notebook

## Loading Frey 2004, Frey 2006 and Liou 2010 Substorm lists
All three of these lists can be obtained from SuperMAG https://supermag.jhuapl.edu/products/ . However, currently the copy of Liou has bugs so the correct version should be obtained from https://agupubs.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1029%2F2010JA015578&file=jgra20697-sup-0002-ds01.pdf.

We assume the user has downloaded the Frey list from SuperMAG and the pdf containing the Liou list from the papers supplementary material.

We have provided code to convert and format the pdf of the substorm list from Liou 2010.

In [14]:
import pandas as pd
from Wind_Waves.reading_tools import lioupdf2csv
frey= pd.read_csv('../Example_Data/Full_Run_Through/frey.csv',
                    parse_dates=['Date_UTC']) # Load Frey (SuperMAG already combines both Frey lists)
liou= pd.read_csv(lioupdf2csv('../Example_Data/Full_Run_Through/liou.pdf'), parse_dates=['Date_UTC'])

## Combine the Frey and Liou lists
Now we must combine the Frey and Liou lists into one combined imagery substorm lists

In [15]:
liou.rename(columns={'mlt':'MLT', 'mlat':'MLAT', 'glon':'GLON', 'glat':'GLAT'}, inplace=True)
combined= pd.concat((liou, frey)).sort_values('Date_UTC')
combined['Date_UTC']= combined['Date_UTC'].values.astype('datetime64[m]').astype('datetime64[ns]')
combined.sort_values('Date_UTC', inplace=True)
combined.index= list(range(len(combined)))
combined.to_csv('../Example_Data/Full_Run_Through/combined_imagery.csv', index=False)

## Load Remaining Lists
The remaining list Newell 2011, Ohtani 2020 and SOPHIE can all be obtained from SuperMAG (https://supermag.jhuapl.edu/products/). SuperMAG regularly updates these lists be rerunning the algorithms when new magnetometer data is uploaded and all these lists will cover the entire operational period of the SML index. We will assume the user has downloaded the lists from SuperMAG in csv format.

In [3]:
import pandas as pd
newell= pd.read_csv('../Example_Data/Full_Run_Through/newell.csv',
                    parse_dates=['Date_UTC'])
ohtani= pd.read_csv('../Example_Data/Full_Run_Through/ohtani.csv',
                    parse_dates=['Date_UTC'])
sophie= pd.read_csv('../Example_Data/Full_Run_Through/sophie.csv',
                    parse_dates=['Date_UTC'])
combined= pd.read_csv('../Example_Data/Full_Run_Through/combined_imagery.csv', parse_dates=['Date_UTC'])

## Create Substorm Epoch
First we reload the akr frequency limits file

In [4]:
akr_flims= pd.read_csv('../Example_Data/Full_Run_Through/akr_flims_working_version.csv', parse_dates=['Date_UTC'])

Now we use the function find closest to obtain the times of substorm closest to the time of each sweep. We the take the difference to get substorm epoch. We also keep the MLT and MLat of the substorm onsets. Columns named combined, newell, ohtani and sophie contain the corresponding substorm epoch value in terms of minutes.

In [5]:
from MatchSubpy.event_match import find_closest
subs= [combined, newell, ohtani, sophie]
sub_names= ['combined', 'newell', 'ohtani', 'sophie']
for df, name in zip(subs, sub_names):
    akr_flims[f'{name}_closest'], ind= find_closest(akr_flims.Date_UTC.values, df.Date_UTC.values, return_index=True)
    akr_flims[f'{name}']= ((akr_flims.Date_UTC- akr_flims[f'{name}_closest']).values.\
        astype('timedelta64[ns]').astype('float64'))/(1e9*60)
    akr_flims[f'{name}_MLT']=  df.MLT.values[ind]
    akr_flims[f'{name}_MLAT']= df.MLAT.values[ind]
akr_flims.to_csv('../Example_Data/Full_Run_Through/akr_substorm_epoch.csv', index=False)